In [2]:
#Import Required Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from flask import Flask, request, render_template
import os
from PIL import Image 


In [3]:
#Data Preprocessing
def preprocess_image(img_path, target_size, preprocess_input_func):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input_func(img_array)


In [4]:
# Load Pre-trained Models and Modify for Binary Classification
def build_model(base_model, preprocess_input_func):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model, preprocess_input_func

# Load models
resnet_base = ResNet50(weights='imagenet', include_top=False)
vgg_base = VGG16(weights='imagenet', include_top=False)
inception_base = InceptionV3(weights='imagenet', include_top=False)

resnet_model, resnet_preprocess_func = build_model(resnet_base, resnet_preprocess)
vgg_model, vgg_preprocess_func = build_model(vgg_base, vgg_preprocess)
inception_model, inception_preprocess_func = build_model(inception_base, inception_preprocess)


In [5]:
#Data Generators and Training
def remove_non_image_files(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            img = Image.open(file_path)
            img.verify()  # Verify that it is, in fact, an image
        except (IOError, SyntaxError) as e:
            print(f'Removing non-image file: {file_path}')
            os.remove(file_path)

# Run this on both beating and non_beating directories
remove_non_image_files(r'C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating')
remove_non_image_files(r'C:\Users\kakan\OneDrive\Desktop\cardiac project\data\non_beating')

# Data augmentation and data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    r'C:\Users\kakan\OneDrive\Desktop\cardiac project\data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\kakan\OneDrive\Desktop\cardiac project\data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')

# Train models
resnet_model.fit(train_generator, validation_data=validation_generator, epochs=10)
vgg_model.fit(train_generator, validation_data=validation_generator, epochs=10)
inception_model.fit(train_generator, validation_data=validation_generator, epochs=10)


Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-1-1.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-1-2.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-10-1.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-11-1.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-11-2,3.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-12-1.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-12-2.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-13-1,2.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-14-1.mp4
Removing non-image file: C:\Users\kakan\OneDrive\Desktop\cardiac project\data\beating\D9-15-1.mp4
Removing non-image

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 132s 132s/step - accuracy: 0.4138 - loss: 0.8706 - val_accuracy: 0.8571 - val_loss: 0.7729
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step - accuracy: 0.8621 - loss: 0.8293 - val_accuracy: 0.2857 - val_loss: 0.7056
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 1.0000 - loss: 9.5252e-04 - val_accuracy: 0.1429 - val_loss: 4.3194
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - accuracy: 1.0000 - loss: 1.4736e-05 - val_accuracy: 0.1429 - val_loss: 12.8189
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step - accuracy: 1.0000 - loss: 2.3014e-05 - val_accuracy: 0.1429 - val_loss: 31.3720
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 1.0000 - loss: 1.7102e-04 - val_accuracy: 0.1429 - val_loss: 59.9754
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - accuracy: 1.0000 - loss: 1.7325e-04 - val_accuracy: 0.1429 - val_loss: 109.9013
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step - accuracy: 1.0000 - loss: 0.0021 - val_accuracy: 0.1429 - v

In [8]:
#Save Models
resnet_model.save('resnet_model.h5')
vgg_model.save('vgg_model.h5')
inception_model.save('inception_model.h5')


In [9]:
#Create Flask App for User Interface
app = Flask(__name__)

# Load models
resnet_model = tf.keras.models.load_model('resnet_model.h5')
vgg_model = tf.keras.models.load_model('vgg_model.h5')
inception_model = tf.keras.models.load_model('inception_model.h5')

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    if file:
        img_path = os.path.join('uploads', file.filename)
        file.save(img_path)

        img_resnet = preprocess_image(img_path, (224, 224), resnet_preprocess_func)
        img_vgg = preprocess_image(img_path, (224, 224), vgg_preprocess_func)
        img_inception = preprocess_image(img_path, (224, 224), inception_preprocess_func)

        prediction_resnet = resnet_model.predict(img_resnet)[0][0]
        prediction_vgg = vgg_model.predict(img_vgg)[0][0]
        prediction_inception = inception_model.predict(img_inception)[0][0]

        # Majority vote
        predictions = [prediction_resnet, prediction_vgg, prediction_inception]
        final_prediction = np.mean(predictions) > 0.5

        result = 'Beating' if final_prediction else 'Non-beating'
        return result

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1